### 경사하강법을 이용한 행렬 분해 이해

In [21]:
import numpy as np

# 원본 행렬 R 생성, 
# 분해 행렬 P와 Q 초기화, 잠재요인 차원 K는 3 설정. 
R = np.array([[4, np.NaN, np.NaN, 2, np.NaN ],
              [np.NaN, 5, np.NaN, 3, 1 ],
              [np.NaN, np.NaN, 3, 4, 4 ],
              [5, 2, 1, 2, np.NaN ]])

print(R.shape)
R # R은 4X5 행렬이다.

(4, 5)


array([[ 4., nan, nan,  2., nan],
       [nan,  5., nan,  3.,  1.],
       [nan, nan,  3.,  4.,  4.],
       [ 5.,  2.,  1.,  2., nan]])

In [23]:
num_users, num_items = R.shape

K=3  # 잠재 요인은 3개

print(num_users) # M
print(num_items) # N

4
5


In [24]:
# P, Q 찾기
# P와 Q 매트릭스의 크기를 지정하고 정규분포를 가진 random한 값으로 입력합니다.
np.random.seed(1)

P = np.random.normal(scale=1./K, size=(num_users, K))  # 4X3 P행렬
Q = np.random.normal(scale=1./K, size=(num_items, K))  # 5X3 Q행렬

# 행렬 P, Q 초기화 상태
print(P,'\n')
print(Q)

[[ 0.54144845 -0.2039188  -0.17605725]
 [-0.35765621  0.28846921 -0.76717957]
 [ 0.58160392 -0.25373563  0.10634637]
 [-0.08312346  0.48736931 -0.68671357]] 

[[-0.1074724  -0.12801812  0.37792315]
 [-0.36663042 -0.05747607 -0.29261947]
 [ 0.01407125  0.19427174 -0.36687306]
 [ 0.38157457  0.30053024  0.16749811]
 [ 0.30028532 -0.22790929 -0.04096341]]


In [25]:
from sklearn.metrics import mean_squared_error

# 실제 행렬 R과 예측 행렬 간 오차(RMSE)를 구하는 함수
# R 행렬에서 비어있지 않은 값 : non_zeros
def get_rmse(R, P, Q, non_zeros):
    error = 0
    # 두개의 분해된 행렬 P와 Q.T의 내적으로 예측 R 행렬 생성
    full_pred_matrix = np.dot(P, Q.T)
    
    # 실제 R 행렬에서 널이 아닌 값의 위치 인덱스 추출하여 실제 R 행렬과 예측 행렬의 RMSE 추출
    x_non_zero_ind = [non_zero[0] for non_zero in non_zeros]
    y_non_zero_ind = [non_zero[1] for non_zero in non_zeros]
    R_non_zeros = R[x_non_zero_ind, y_non_zero_ind]
    full_pred_matrix_non_zeros = full_pred_matrix[x_non_zero_ind, y_non_zero_ind]
      
    mse = mean_squared_error(R_non_zeros, full_pred_matrix_non_zeros)
    rmse = np.sqrt(mse)
    
    return rmse

#### 경사하강법에 기반하여 P와 Q 원소들을 업데이트 수행

In [15]:
# R > 0 인 행 위치, 열 위치, 값을 non_zeros 리스트에 저장. 
non_zeros = [ (i, j, R[i,j]) for i in range(num_users) for j in range(num_items) if R[i,j] > 0 ]
non_zeros

[(0, 0, 4.0),
 (0, 3, 2.0),
 (1, 1, 5.0),
 (1, 3, 3.0),
 (1, 4, 1.0),
 (2, 2, 3.0),
 (2, 3, 4.0),
 (2, 4, 4.0),
 (3, 0, 5.0),
 (3, 1, 2.0),
 (3, 2, 1.0),
 (3, 3, 2.0)]

In [26]:
R

array([[ 4., nan, nan,  2., nan],
       [nan,  5., nan,  3.,  1.],
       [nan, nan,  3.,  4.,  4.],
       [ 5.,  2.,  1.,  2., nan]])

## 경사하강법

In [28]:
steps=10000
learning_rate=0.01
r_lambda=0.01

# P와 Q 매트릭스를 계속 업데이트(확률적 경사하강법)
for step in range(steps):  # 1000회 업데이트
    for i, j, r in non_zeros:
        
        # 실제 값과 예측 값의 차이인 오류 값 구함
        eij = r - np.dot(P[i, :], Q[j, :].T)
        
        # Regularization을 반영한 SGD(확률적 경사하강법) 업데이트 공식 적용
        P[i,:] = P[i,:] + learning_rate * ( eij * Q[j,:] - r_lambda*P[i,:] )
        Q[j,:] = Q[j,:] + learning_rate * ( eij * P[i,:] - r_lambda*Q[j,:] )

    rmse = get_rmse(R, P, Q, non_zeros)
    if (step % 50) == 0 :
        print("### iteration step : ", step," rmse : ", rmse)

### iteration step :  0  rmse :  0.016410959588961445
### iteration step :  50  rmse :  0.01637308966218993
### iteration step :  100  rmse :  0.016333882336881784
### iteration step :  150  rmse :  0.016293661274599174
### iteration step :  200  rmse :  0.01625272572302336
### iteration step :  250  rmse :  0.016211338984681307
### iteration step :  300  rmse :  0.016169725523756753
### iteration step :  350  rmse :  0.016128072491623933
### iteration step :  400  rmse :  0.016086533303745463
### iteration step :  450  rmse :  0.016045231961723017
### iteration step :  500  rmse :  0.016004267423347165
### iteration step :  550  rmse :  0.015963717671935662
### iteration step :  600  rmse :  0.01592364333325902
### iteration step :  650  rmse :  0.015884090797210332
### iteration step :  700  rmse :  0.01584509485896164
### iteration step :  750  rmse :  0.015806680922219037
### iteration step :  800  rmse :  0.01576886681829548
### iteration step :  850  rmse :  0.015731664296432853


### iteration step :  7550  rmse :  0.013776602356221078
### iteration step :  7600  rmse :  0.01377196826375449
### iteration step :  7650  rmse :  0.013767391822029278
### iteration step :  7700  rmse :  0.013762872176581498
### iteration step :  7750  rmse :  0.013758408487991588
### iteration step :  7800  rmse :  0.0137539999315711
### iteration step :  7850  rmse :  0.013749645697062225
### iteration step :  7900  rmse :  0.01374534498832819
### iteration step :  7950  rmse :  0.013741097023079494
### iteration step :  8000  rmse :  0.013736901032575848
### iteration step :  8050  rmse :  0.013732756261356473
### iteration step :  8100  rmse :  0.013728661966969202
### iteration step :  8150  rmse :  0.013724617419703945
### iteration step :  8200  rmse :  0.013720621902338151
### iteration step :  8250  rmse :  0.013716674709881046
### iteration step :  8300  rmse :  0.013712775149333204
### iteration step :  8350  rmse :  0.013708922539439035
### iteration step :  8400  rmse : 

In [29]:
pred_matrix = np.dot(P, Q.T)
print('예측 행렬:\n', np.round(pred_matrix, 3))

예측 행렬:
 [[3.991 1.974 1.106 1.997 1.563]
 [4.097 4.978 1.113 2.987 1.005]
 [4.932 2.581 2.988 3.98  3.984]
 [4.974 2.001 1.003 2.002 1.555]]


In [30]:
# 원본 행렬 R
R

array([[ 4., nan, nan,  2., nan],
       [nan,  5., nan,  3.,  1.],
       [nan, nan,  3.,  4.,  4.],
       [ 5.,  2.,  1.,  2., nan]])